In [71]:
import logging
import time
from concurrent.futures import ThreadPoolExecutor

import requests as rq
import pandas as pd

In [72]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [73]:
# лист категорий (блоков) для статей в Ленте
lenta_blocs_indexes = [1,   # Общество/Россия
                       3,   # Бывший СССР
                       4,   # Экономика
                       5,   # Наука и техника
                       8,   # Спорт
                       37,  # Силовые структуры
                       48,  # Туризм/Путешествия
                       87]  # Забота о себе

Ниже представлены вспомогательные классы: ParamsDictType для параметров поиска и абстрактный класс для парсеров NewsParser. Изначально хотел использовать несколько источников, но потом отказался от этой идеи.

In [74]:
from abc import ABC, abstractmethod
from typing import TypedDict, NotRequired


class ParamsDictType(TypedDict):
    size: int
    search_from: int
    bloc: NotRequired[int]
    query: NotRequired[str]


class NewsParser(ABC):
    @abstractmethod
    def parse_category(self, bloc: int = None, query: str = None) -> pd.DataFrame:
        pass

    @abstractmethod
    def parse(self) -> pd.DataFrame:
        pass

    @abstractmethod
    def _get_page_data(self, url: str) -> pd.DataFrame:
        pass


In [75]:
class LentaParser(NewsParser):
    @staticmethod
    def _get_url(params: ParamsDictType) -> str:
        base_url = (
            f"https://lenta.ru/search/v2/process?size={params['size']}&from={params['search_from']}"
            f"&sort=2&domain=1&modified,format=yyyy-MM-dd&modified,from=2023-01-01&type=1"
        )

        if 'query' in params and params['query']:
            base_url += f"&query={params['query']}"
        if 'bloc' in params and params['bloc']:
            base_url += f"&bloc={params['bloc']}"

        return base_url

    def _get_page_data(self, url: str) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей по текущему url
        """
        try:
            time.sleep(2)
            response = rq.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()
            if 'matches' in data:
                return pd.DataFrame(data['matches'])

            logging.warning(f"Статьи не найдены: {url}")
            return pd.DataFrame()
        except Exception as e:
            logging.error(f"Ошибка при загрузке url {url}: {e}")
            return pd.DataFrame()

    def parse_category(self, bloc: int = None, query: str = None) -> pd.DataFrame:
        """
        Парсит переданную категорию и возвращает pd.DataFrame
        """
        size = 250  # скачиванием батчами, чтобы не вылететь по таймауту
        search_from = 1
        articles_df = pd.DataFrame()

        while search_from < 2500:
            params = {
                'size': size,
                'search_from': search_from,
            }
            if query:
                params['query'] = query
            if bloc:
                params['bloc'] = bloc

            url = self._get_url(params)
            page_data = self._get_page_data(url)

            if page_data.empty:
                logging.info(f"Не найдено данных для категории {bloc}")
                break

            articles_df = pd.concat([articles_df, page_data], ignore_index=True)
            search_from += size

        logging.info(f"Для категории {bloc} найдено {articles_df.shape[0]} статей")
        return articles_df

    def parse(self) -> pd.DataFrame:
        """
        Парсит все переданные категории и возвращает обший pd.DataFrame
        """
        df = pd.DataFrame()

        with ThreadPoolExecutor() as executor:
            results = list(executor.map(self.parse_category, lenta_blocs_indexes))

        for result in results:
            df = pd.concat([df, result], ignore_index=True)

        # Тема "Строительство" парсится отдельно через query, так как такого отдельного блока тем на Ленте нет
        query_results = self.parse_category(query='Строительство')
        query_results['bloc'] = 6
        df = pd.concat([df, query_results], ignore_index=True)

        logging.info(f"Всего статей: {df.shape[0]}")
        return df

In [76]:
lenta_parser = LentaParser()
lenta_df = lenta_parser.parse()
lenta_df.shape

2024-12-22 16:47:26,238 - INFO - Для категории 3 найдено 2500 статей
2024-12-22 16:47:27,055 - INFO - Для категории 37 найдено 2500 статей
2024-12-22 16:47:27,242 - INFO - Для категории 8 найдено 2500 статей
2024-12-22 16:47:29,014 - INFO - Для категории 48 найдено 2500 статей
2024-12-22 16:47:29,316 - INFO - Для категории 4 найдено 2500 статей
2024-12-22 16:47:29,895 - INFO - Для категории 1 найдено 2500 статей
2024-12-22 16:47:30,585 - INFO - Для категории 87 найдено 2500 статей
2024-12-22 16:47:30,767 - INFO - Для категории 5 найдено 2500 статей
2024-12-22 16:48:14,691 - INFO - Для категории None найдено 2500 статей
2024-12-22 16:48:14,702 - INFO - Всего статей: 22500


(22500, 16)

In [77]:
lenta_df.head()

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1754015,https://lenta.ru/news/2024/12/22/on-pozhaleet-...,"«Он пожалеет о том, что пытается сделать». Пут...",1734870420,1734870889,1,1,0,0,1,[1],https://icdn.lenta.ru/images/2024/12/22/14/202...,1734870420,Фото: Александр Казаков / РИА Новости Варвара ...,"«Он пожалеет о том, что пытается сделать». Пут...",Фото: Александр Казаков / РИА Новости ... к гл...
1,1754016,https://lenta.ru/news/2024/12/22/putin-otvetil...,Путин ответил на вопрос о судьбе подаренных бо...,1734869160,1734869551,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2024/12/22/15/202...,1734869160,Фото: Maxim Shemetov / Reuters Полина Кислицын...,Путин ответил на вопрос о судьбе подаренных бо...,Фото: Maxim Shemetov / Reuters Полина ... теле...
2,1754012,https://lenta.ru/news/2024/12/22/putin-dal-kom...,Путин дал команду к запуску новых автотранспор...,1734867720,1734868487,1,1,0,0,1,[1],https://icdn.lenta.ru/images/2024/12/22/14/202...,1734867720,Фото: Максим Богодвид / РИА Новости Екатерина ...,Путин дал команду к запуску новых автотранспор...,Фото: Максим Богодвид / РИА Новости ... новых ...
3,1754000,https://lenta.ru/news/2024/12/22/rossiyanin-re...,Россиянин решил отомстить возлюбленной за разр...,1734866700,1734867056,1,1,0,0,1,[4],https://icdn.lenta.ru/images/2024/12/22/14/202...,1734866700,Фото: Максим Богодвид / РИА Новости Полина Кис...,Россиянин решил отомстить возлюбленной за разр...,Фото: Максим Богодвид / РИА Новости ... и выби...
4,1754006,https://lenta.ru/news/2024/12/22/putin-zayavil...,Путин заявил о глубокой погруженности в разраб...,1734866700,1734868245,1,1,0,0,1,[1],https://icdn.lenta.ru/images/2024/12/22/14/202...,1734866700,Фото: Пресс-служба Минобороны РФ / РИА Новости...,Путин заявил о глубокой погруженности в разраб...,Фото: Пресс-служба Минобороны РФ / РИА ... рак...


In [78]:
lenta_df['bloc'].value_counts(normalize=True)

bloc
1     0.111111
3     0.111111
4     0.111111
5     0.111111
8     0.111111
37    0.111111
48    0.111111
87    0.111111
6     0.111111
Name: proportion, dtype: float64

In [79]:
df = lenta_df[['bloc', 'text']] # оставляем только признак текст и таргет тему (блок)
df = df[df['text'].notnull() & (df['text'].str.split().str.len() > 5)] # удаляем короткие и отсутствующие тексты
df = df.drop_duplicates() # удаляем дубликаты

df.shape

(22500, 2)

In [80]:
# Маппим индексы тем с сайта Ленты с названиями
lenta_categories = {
    1: 'Общество/Россия',
    3: 'Бывший СССР',
    4: 'Экономика',
    5: 'Наука и техника',
    8: 'Спорт',
    37: 'Силовые структуры',
    48: 'Туризм/Путешествия',
    87: 'Забота о себе',
    6: 'Строительство'
}

# Маппим индексы тем из задания с названиями
article_categories = {
    0: 'Общество/Россия',
    1: 'Экономика',
    2: 'Силовые структуры',
    3: 'Бывший СССР',
    4: 'Спорт',
    5: 'Забота о себе',
    6: 'Строительство',
    7: 'Туризм/Путешествия',
    8: 'Наука и техника'
}

new_mapping = {v: k for k, v in article_categories.items()}

# Подменяем индексы тем на нужные по заданию
df['bloc'] = df['bloc'].dropna().map(lenta_categories).map(new_mapping)
df['bloc'].value_counts(normalize=True)

bloc
0    0.111111
3    0.111111
1    0.111111
8    0.111111
4    0.111111
2    0.111111
7    0.111111
5    0.111111
6    0.111111
Name: proportion, dtype: float64

In [81]:
df.to_csv('./lenta_df.csv')